In [13]:
# 📦 라이브러리 로딩
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

# ✅ 한글 설정 및 마이너스 깨짐 방지
plt.rc('font', family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

# 📂 데이터 불러오기
train = pd.read_csv('train.csv').drop(columns=['ID'])
test = pd.read_csv('test.csv').drop(columns=['ID'])
sample_submission = pd.read_csv('sample_submission.csv')

# 🔧 범위값 처리 함수
def convert_range_to_float(value):
    if isinstance(value, str) and '-' in value:
        try:
            low, high = map(float, value.split('-'))
            return (low + high) / 2
        except:
            return np.nan
    try:
        return float(value)
    except:
        return np.nan

# 🔧 범주형 인코딩
def encode_categoricals(df, cols):
    df = df.copy()
    for col in cols:
        df[col] = LabelEncoder().fit_transform(df[col].astype(str))
    return df

# 🔧 결측치 처리 함수
def fill_missing_values(df, is_train=True):
    df = df.copy()

    # 범위 문자열 → 평균 숫자 처리
    for col in ['연매출(억원)', '총 투자금(억원)', '기업가치(백억원)']:
        df[col] = df[col].apply(convert_range_to_float)

    # 분야 결측 및 인코딩
    if '분야' in df.columns:
        df['분야'] = df['분야'].fillna('Unknown')
        df['분야'] = LabelEncoder().fit_transform(df['분야'])

    # 국가, 투자단계 인코딩
    df = encode_categoricals(df, ['국가', '투자단계'])

    # ✅ 결측 플래그 추가 함수
    def add_missing_flag(column):
        flag_col = f'{column}_결측'
        df[flag_col] = df[column].isnull().astype(int)

    # ✅ 피처셋 생성 함수
    def get_features(base):
        return base + (['성공확률'] if is_train else [])

    # 1. 직원 수
    if '직원 수' in df.columns:
        add_missing_flag('직원 수')
        features = get_features(['설립연도', '국가', '투자단계', '연매출(억원)', '총 투자금(억원)', 'SNS 팔로워 수(백만명)'])
        complete = df[df['직원 수'].notnull()]
        missing = df[df['직원 수'].isnull()]
        if not complete.empty and not missing.empty:
            model = GradientBoostingRegressor()
            model.fit(complete[features], complete['직원 수'])
            df.loc[df['직원 수'].isnull(), '직원 수'] = model.predict(missing[features])

    # 2. 고객 수
    if '고객수(백만명)' in df.columns:
        add_missing_flag('고객수(백만명)')
        features = get_features(['설립연도', '직원 수', '분야', '연매출(억원)', '총 투자금(억원)', 'SNS 팔로워 수(백만명)'])
        complete = df[df['고객수(백만명)'].notnull()]
        missing = df[df['고객수(백만명)'].isnull()]
        if not complete.empty and not missing.empty:
            model = GradientBoostingRegressor()
            model.fit(complete[features], complete['고객수(백만명)'])
            df.loc[df['고객수(백만명)'].isnull(), '고객수(백만명)'] = model.predict(missing[features])

    # 3. 기업가치
    if '기업가치(백억원)' in df.columns:
        add_missing_flag('기업가치(백억원)')
        features = get_features(['설립연도', '직원 수', '고객수(백만명)', '분야', '연매출(억원)', '총 투자금(억원)', 'SNS 팔로워 수(백만명)'])
        complete = df[df['기업가치(백억원)'].notnull()]
        missing = df[df['기업가치(백억원)'].isnull()]
        if not complete.empty and not missing.empty:
            model = GradientBoostingRegressor()
            model.fit(complete[features], complete['기업가치(백억원)'])
            df.loc[df['기업가치(백억원)'].isnull(), '기업가치(백억원)'] = model.predict(missing[features])

    return df

# 📌 데이터 전처리
train_filled = fill_missing_values(train, is_train=True)
test_filled = fill_missing_values(test, is_train=False)

# 📌 이상치 처리 함수
def process_outliers(train_df, test_df, num_cols, method='flag+clip'):
    train, test = train_df.copy(), test_df.copy()
    for col in num_cols:
        Q1, Q3 = train[col].quantile([0.25, 0.75])
        IQR = Q3 - Q1
        lower, upper = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR

        if 'flag' in method:
            train[f'{col}_이상치여부'] = ((train[col] < lower) | (train[col] > upper)).astype(int)
            test[f'{col}_이상치여부'] = ((test[col] < lower) | (test[col] > upper)).astype(int)

        if 'clip' in method:
            train[col] = train[col].clip(lower, upper)
            test[col] = test[col].clip(lower, upper)

    return train, test

num_cols = train_filled.select_dtypes(include=np.number).columns.drop('성공확률').tolist()
train_processed, test_processed = process_outliers(train_filled, test_filled, num_cols)



def detect_outliers_summary(df, columns):
    summary = []
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
        summary.append({
            '컬럼명': col,
            '이상치 수': len(outliers),
            '전체 대비 비율(%)': round(len(outliers) / len(df) * 100, 2)
        })
    return pd.DataFrame(summary).sort_values(by='이상치 수', ascending=False)

# log 변환 (0보다 큰 값만 변환, log1p는 log(1+x))
num_cols = train_filled.select_dtypes(include=['int64', 'float64']).columns.tolist()
print(num_cols)
log_train = train_filled[num_cols].copy()
for col in num_cols:
    if (log_train[col] > 0).all():  # 음수, 0 있는 컬럼은 제외
        log_train[col] = np.log1p(log_train[col])

def process_outliers_train_test(train_df, test_df, num_cols, method='flag+clip'):
    """
    train 데이터 기준으로 IQR 이상치 탐지 기준을 잡고,
    train/test 모두 동일한 방식으로 이상치 처리하는 함수
    """
    train_processed = train_df.copy()
    test_processed = test_df.copy()
    outlier_bounds = {}

    for col in num_cols:
        Q1 = train_df[col].quantile(0.25)
        Q3 = train_df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        outlier_bounds[col] = (lower, upper)

        # 이상치 플래그
        if 'flag' in method:
            train_processed[f'{col}_이상치여부'] = ((train_df[col] < lower) | (train_df[col] > upper)).astype(int)
            test_processed[f'{col}_이상치여부'] = ((test_df[col] < lower) | (test_df[col] > upper)).astype(int)

        # 클리핑
        if 'clip' in method:
            train_processed[col] = train_df[col].clip(lower, upper)
            test_processed[col] = test_df[col].clip(lower, upper)

        # 로그 변환
        if 'log' in method:
            if (train_processed[col] >= 0).all() and (test_processed[col] >= 0).all():
                train_processed[col] = np.log1p(train_processed[col])
                test_processed[col] = np.log1p(test_processed[col])
            else:
                print(f"[경고] {col}은 log1p 불가능 (음수 또는 0 포함)")

    return train_processed, test_processed, outlier_bounds

# '성공확률'은 train에만 있으므로 제외
num_cols = train_filled.select_dtypes(include=np.number).columns.tolist()
num_cols = [col for col in num_cols if col != '성공확률']

# 이상치 처리 수행
train_processed, test_processed, bounds = process_outliers_train_test(train_filled, test_filled, num_cols, method='flag+clip')

# 결과 확인: 일부 컬럼에 대해 이상치 여부 플래그 분포 출력
for col in ['연매출(억원)', '고객수(백만명)', 'SNS 팔로워 수(백만명)']:
    flag_col = f'{col}_이상치여부'
    if flag_col in train_processed.columns:
        print(f"▶ {flag_col} - 이상치 개수(train): {train_processed[flag_col].sum()}")

print("✅ train 결측 확인:\n", train_filled.isnull().sum().sort_values(ascending=False))
print("✅ test 결측 확인:\n", test_filled.isnull().sum().sort_values(ascending=False))

['설립연도', '국가', '분야', '투자단계', '직원 수', '고객수(백만명)', '총 투자금(억원)', '연매출(억원)', 'SNS 팔로워 수(백만명)', '기업가치(백억원)', '성공확률', '직원 수_결측', '고객수(백만명)_결측', '기업가치(백억원)_결측']
▶ 연매출(억원)_이상치여부 - 이상치 개수(train): 0
▶ 고객수(백만명)_이상치여부 - 이상치 개수(train): 0
▶ SNS 팔로워 수(백만명)_이상치여부 - 이상치 개수(train): 0
✅ train 결측 확인:
 설립연도              0
국가                0
분야                0
투자단계              0
직원 수              0
인수여부              0
상장여부              0
고객수(백만명)          0
총 투자금(억원)         0
연매출(억원)           0
SNS 팔로워 수(백만명)    0
기업가치(백억원)         0
성공확률              0
직원 수_결측           0
고객수(백만명)_결측       0
기업가치(백억원)_결측      0
dtype: int64
✅ test 결측 확인:
 설립연도              0
국가                0
분야                0
투자단계              0
직원 수              0
인수여부              0
상장여부              0
고객수(백만명)          0
총 투자금(억원)         0
연매출(억원)           0
SNS 팔로워 수(백만명)    0
기업가치(백억원)         0
직원 수_결측           0
고객수(백만명)_결측       0
기업가치(백억원)_결측      0
dtype: int64


In [11]:
# 📌 파생변수 생성
def create_features(df):
    df = df.copy()

    return df

X = train_processed.copy()  # ← 여기서 train_processed에 이상치 플래그가 들어있어야 함
X = create_features(X)      # 파생변수 추가
X_test = test_processed.copy()
X_test = create_features(X_test)
y = train_processed['성공확률']  # 또는 미리 분리한 y 사용

# 📌 필요없는 컬럼 제거
remove_cols = ['직원 수', '연매출(억원)', '설립연도', '총 투자금(억원)', '직원 수_결측', '고객수_결측']
X.drop(columns=[col for col in remove_cols if col in X.columns], inplace=True)
X_test.drop(columns=[col for col in remove_cols if col in X_test.columns], inplace=True)

# ✅ 이진형 인코딩
for col in ['인수여부', '상장여부']:
    for df in [X, X_test]:
        if col in df.columns:
            df[col] = df[col].map({'No': 0, 'Yes': 1})

# '성공확률' 컬럼이 있다면 삭제
if '성공확률' in X.columns:
    X = X.drop(columns=['성공확률'])

# y 결합
X_with_y = X.join(y.rename("성공확률"))  

In [27]:
from autofeat import AutoFeatRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
import pandas as pd
import numpy as np

# 1. 데이터 분할
X_train_af, X_val_af, y_train_af, y_val_af = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. AutoFeatRegressor 선언
afr = AutoFeatRegressor(verbose=1, feateng_steps=2)

try:
    # 3. 파생변수 생성
    X_train_transformed = afr.fit_transform(X_train_af, y_train_af)
    X_train_transformed = pd.DataFrame(X_train_transformed)
    
    # ✅ NaN 또는 Inf 존재 확인
    bad_cols = X_train_transformed.columns[
        X_train_transformed.isnull().any() | np.isinf(X_train_transformed).any()
    ]
    print("🧹 제거할 문제 있는 컬럼 수:", len(bad_cols))

    # ✅ 문제 있는 열 제거
    X_train_transformed = X_train_transformed.drop(columns=bad_cols)

    # 4. val transform 후 같은 컬럼만 추출
    X_val_transformed = afr.transform(X_val_af)
    X_val_transformed = pd.DataFrame(X_val_transformed)
    X_val_transformed = X_val_transformed[X_train_transformed.columns]
    X_val_transformed = X_val_transformed.replace([np.inf, -np.inf], 0).fillna(0)

    # 5. XGBoost 학습 및 평가
    model = XGBRegressor(n_estimators=500, learning_rate=0.01, max_depth=5, random_state=42)
    model.fit(X_train_transformed, y_train_af)
    y_pred = model.predict(X_val_transformed)
    mae = mean_absolute_error(y_val_af, y_pred)
    print(f"📉 Validation MAE: {mae:.5f}")

except Exception as e:
    print("❌ 최종 에러:", e)


2025-05-01 13:29:57,534 INFO: [AutoFeat] The 2 step feature engineering process could generate up to 13041 features.
2025-05-01 13:29:57,535 INFO: [AutoFeat] With 3500 data points this new feature matrix would use about 0.18 gb of space.
2025-05-01 13:29:57,538 INFO: [feateng] Step 1: transformation of original features


2025-05-01 13:29:59,400 INFO: [feateng] Generated 25 transformed features from 23 original features - done.
2025-05-01 13:29:59,403 INFO: [feateng] Step 2: first combination of features


c:\Users\human\.conda\envs\dacon\Lib\site-packages\numpy\_core\_methods.py:205: RuntimeWarning: overflow encountered in reduce
2025-05-01 13:30:00,217 INFO: [feateng] Generated 614 feature combinations from 1128 original feature tuples - done.
2025-05-01 13:30:00,241 INFO: [feateng] Generated altogether 1148 new features in 2 steps
2025-05-01 13:30:00,242 INFO: [feateng] Removing correlated features, as well as additions at the highest level
2025-05-01 13:30:00,357 INFO: [feateng] Generated a total of 470 additional features


[featsel] Scaling data.../           1128 feature tuples combined

2025-05-01 13:30:00,530 INFO: [featsel] Feature selection run 1/5


done.
❌ 최종 에러: Input X contains NaN.
LassoLarsCV does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values
